In [206]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

In [207]:
base_url ='https://api.fxratesapi.com'

*https://fxratesapi.com/docs/endpoints/list-available-currencies*

In [208]:
def get_currencies():
  df_currencies = pd.DataFrame()
  r = requests.get(base_url+'/currencies')
  currencies = r.json()
  for key, value in currencies.items():
    df = pd.json_normalize(value)
    df_currencies = pd.concat([df_currencies, df], ignore_index=True)
  return df_currencies

df_currencies = get_currencies()
print(list(df_currencies['code']))
# df_currencies = df_currencies[df_currencies['code'].isin(['USD', 'ARS', 'BTC', 'ETH'])].reset_index()
df_currencies = df_currencies[df_currencies['code'].isin(['USD'])].reset_index()
df_currencies

['AFN', 'ALL', 'AMD', 'ANG', 'AOA', 'ARS', 'AUD', 'AWG', 'AZN', 'BAM', 'BBD', 'BDT', 'BGN', 'BHD', 'BIF', 'BMD', 'BND', 'BOB', 'BRL', 'BSD', 'BWP', 'BYN', 'BYR', 'BZD', 'CAD', 'CDF', 'CHF', 'CLF', 'CLP', 'CNY', 'COP', 'CRC', 'CUC', 'CUP', 'CVE', 'CZK', 'DJF', 'DKK', 'DOP', 'DZD', 'ERN', 'ETB', 'EUR', 'FJD', 'FKP', 'GBP', 'GEL', 'GGP', 'GHS', 'GIP', 'GMD', 'GNF', 'GTQ', 'GYD', 'HKD', 'HNL', 'HRK', 'HTG', 'HUF', 'IDR', 'ILS', 'IMP', 'IQD', 'IRR', 'ISK', 'JEP', 'JMD', 'JOD', 'JPY', 'KES', 'KGS', 'KHR', 'KMF', 'KPW', 'KRW', 'KWD', 'KYD', 'KZT', 'LAK', 'LBP', 'LKR', 'LRD', 'LSL', 'LTL', 'LVL', 'MAD', 'MDL', 'XAG', 'XAU', 'XCD', 'XDR', 'XOF', 'XPF', 'YER', 'ZAR', 'ZMK', 'ZMW', 'ZWL', 'XPT', 'XPD', 'BTC', 'ETH', 'BNB', 'XRP', 'SOL', 'DOT', 'AVAX', 'MATIC', 'LTC', 'ADA', 'USDT', 'USDC', 'DAI', 'BUSD', 'ARB', 'OP', 'AED', 'BTN', 'MKD', 'MMK', 'MNT', 'MOP', 'MRO', 'MUR', 'MVR', 'MWK', 'MXN', 'MZN', 'NAD', 'NGN', 'NIO', 'NOK', 'NPR', 'NZD', 'OMR', 'PAB', 'PEN', 'PGK', 'PHP', 'PKR', 'PLN', 'PYG', 

,index,code,name,decimal_digits,name_plural,rounding,symbol,symbol_native
0,171,USD,US Dollar,2,US dollars,0,$,$


In [209]:
start_date = datetime(datetime.now().year, 1, 1, tzinfo=timezone.utc)
end_date = datetime.now(timezone.utc)
formato = "%Y-%m-%dT%H:%M:%S.%fZ"
start_date = start_date.strftime(formato)
end_date = end_date.strftime(formato)
print({'start_date':start_date, 'end_date':end_date})

{'start_date': '2023-01-01T00:00:00.000000Z', 'end_date': '2023-12-03T23:06:36.473188Z'}


*https://fxratesapi.com/docs/endpoints/time-series-exchange-rates*

In [210]:
def get_series_times(row):
  base_code = row['code']
  query = {
      'base': base_code,
      'start_date': start_date,
      'end_date': end_date,
      'currencies': ['USD', 'ARS', 'BTC', 'ETH', 'BUSD', 'USDC', 'USDT', 'ADA', 'BNB', 'EUR', 'BRL'],
  }
  r = requests.get(base_url+'/timeseries', params=query)
  time_series = r.json()
  return (
      time_series['base'] if 'base' in time_series else "N/A",
      time_series['start_date'] if 'start_date' in time_series else "N/A",
      time_series['end_date'] if 'end_date' in time_series else "N/A",
      time_series['rates'] if 'rates' in time_series else "N/A"
  )

In [211]:
df_time_series = pd.DataFrame()
df_time_series['base'], df_time_series['start_date'], df_time_series['end_date'], df_time_series['rates'] = zip(*df_currencies.apply(get_series_times, axis=1))
df_time_series

,base,start_date,end_date,rates
0,USD,2023-01-01T00:00:00.000Z,2023-12-03T23:06:36.473Z,{'2023-12-02T23:59:00.000Z': {'ADA': 2.5115150...


In [212]:
df_time_series_values = pd.DataFrame()

for index, row in df_time_series.iterrows():
  base_code = row['base']
  for key, value in row['rates'].items():
    df = pd.json_normalize(value)
    df['code'] = base_code
    df['date'] = key
    df_time_series_values = pd.concat([df_time_series_values, df], ignore_index=True)

df_time_series_values

,ADA,ARS,BNB,BRL,BTC,ETH,EUR,USD,code,date
0,2.511515,361.095970,0.004230,4.879361,0.000025,0.000461,0.918910,1.0,USD,2023-12-02T23:59:00.000Z
1,2.602518,361.095967,0.004246,4.879360,0.000026,0.000479,0.918910,1.0,USD,2023-12-01T23:59:00.000Z
2,2.648441,360.455846,0.004222,4.920221,0.000026,0.000486,0.918120,1.0,USD,2023-11-30T23:59:00.000Z
3,2.616139,360.019446,0.004235,4.901851,0.000026,0.000492,0.911350,1.0,USD,2023-11-29T23:59:00.000Z
4,2.597127,359.467741,0.004238,4.872911,0.000026,0.000488,0.908880,1.0,USD,2023-11-28T23:59:00.000Z
...,...,...,...,...,...,...,...,...,...,...
293,3.705081,178.909824,0.003890,5.351306,0.000059,0.000798,0.950401,NaN,USD,2023-01-05T23:59:00.000Z
294,3.743071,178.571917,0.003877,5.431609,0.000059,0.000796,0.942652,NaN,USD,2023-01-04T23:59:00.000Z
295,3.941406,178.380409,0.004043,5.480010,0.000060,0.000821,0.948182,NaN,USD,2023-01-03T23:59:00.000Z
296,3.940376,178.152844,0.004063,5.363887,0.000060,0.000823,0.936562,NaN,USD,2023-01-02T23:59:00.000Z


In [213]:
df_final = df_currencies.merge(df_time_series_values, on='code')
df_final

,index,code,name,decimal_digits,name_plural,rounding,symbol,symbol_native,ADA,ARS,BNB,BRL,BTC,ETH,EUR,USD,date
0,171,USD,US Dollar,2,US dollars,0,$,$,2.511515,361.095970,0.004230,4.879361,0.000025,0.000461,0.918910,1.0,2023-12-02T23:59:00.000Z
1,171,USD,US Dollar,2,US dollars,0,$,$,2.602518,361.095967,0.004246,4.879360,0.000026,0.000479,0.918910,1.0,2023-12-01T23:59:00.000Z
2,171,USD,US Dollar,2,US dollars,0,$,$,2.648441,360.455846,0.004222,4.920221,0.000026,0.000486,0.918120,1.0,2023-11-30T23:59:00.000Z
3,171,USD,US Dollar,2,US dollars,0,$,$,2.616139,360.019446,0.004235,4.901851,0.000026,0.000492,0.911350,1.0,2023-11-29T23:59:00.000Z
4,171,USD,US Dollar,2,US dollars,0,$,$,2.597127,359.467741,0.004238,4.872911,0.000026,0.000488,0.908880,1.0,2023-11-28T23:59:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,171,USD,US Dollar,2,US dollars,0,$,$,3.705081,178.909824,0.003890,5.351306,0.000059,0.000798,0.950401,NaN,2023-01-05T23:59:00.000Z
294,171,USD,US Dollar,2,US dollars,0,$,$,3.743071,178.571917,0.003877,5.431609,0.000059,0.000796,0.942652,NaN,2023-01-04T23:59:00.000Z
295,171,USD,US Dollar,2,US dollars,0,$,$,3.941406,178.380409,0.004043,5.480010,0.000060,0.000821,0.948182,NaN,2023-01-03T23:59:00.000Z
296,171,USD,US Dollar,2,US dollars,0,$,$,3.940376,178.152844,0.004063,5.363887,0.000060,0.000823,0.936562,NaN,2023-01-02T23:59:00.000Z


In [214]:
df_final[df_final.date.str.contains('2023-12')]

,index,code,name,decimal_digits,name_plural,rounding,symbol,symbol_native,ADA,ARS,BNB,BRL,BTC,ETH,EUR,USD,date
0,171,USD,US Dollar,2,US dollars,0,$,$,2.511515,361.095970,0.004230,4.879361,0.000025,0.000461,0.91891,1.0,2023-12-02T23:59:00.000Z
1,171,USD,US Dollar,2,US dollars,0,$,$,2.602518,361.095967,0.004246,4.879360,0.000026,0.000479,0.91891,1.0,2023-12-01T23:59:00.000Z


In [215]:
df_final[df_final.code == 'USD']

,index,code,name,decimal_digits,name_plural,rounding,symbol,symbol_native,ADA,ARS,BNB,BRL,BTC,ETH,EUR,USD,date
0,171,USD,US Dollar,2,US dollars,0,$,$,2.511515,361.095970,0.004230,4.879361,0.000025,0.000461,0.918910,1.0,2023-12-02T23:59:00.000Z
1,171,USD,US Dollar,2,US dollars,0,$,$,2.602518,361.095967,0.004246,4.879360,0.000026,0.000479,0.918910,1.0,2023-12-01T23:59:00.000Z
2,171,USD,US Dollar,2,US dollars,0,$,$,2.648441,360.455846,0.004222,4.920221,0.000026,0.000486,0.918120,1.0,2023-11-30T23:59:00.000Z
3,171,USD,US Dollar,2,US dollars,0,$,$,2.616139,360.019446,0.004235,4.901851,0.000026,0.000492,0.911350,1.0,2023-11-29T23:59:00.000Z
4,171,USD,US Dollar,2,US dollars,0,$,$,2.597127,359.467741,0.004238,4.872911,0.000026,0.000488,0.908880,1.0,2023-11-28T23:59:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,171,USD,US Dollar,2,US dollars,0,$,$,3.705081,178.909824,0.003890,5.351306,0.000059,0.000798,0.950401,NaN,2023-01-05T23:59:00.000Z
294,171,USD,US Dollar,2,US dollars,0,$,$,3.743071,178.571917,0.003877,5.431609,0.000059,0.000796,0.942652,NaN,2023-01-04T23:59:00.000Z
295,171,USD,US Dollar,2,US dollars,0,$,$,3.941406,178.380409,0.004043,5.480010,0.000060,0.000821,0.948182,NaN,2023-01-03T23:59:00.000Z
296,171,USD,US Dollar,2,US dollars,0,$,$,3.940376,178.152844,0.004063,5.363887,0.000060,0.000823,0.936562,NaN,2023-01-02T23:59:00.000Z
